# ChineseWordSegmentation - FinalModel (2 Machine)
Final training for the delivery of the homework

After having perfomed the training with dataset MSR and PKU, we realized that the test could be done with any of the data in the datasets availabe, so we will train a model with a parte of the complete datasets and try to come up with a general network representation to predict the labels accordingly.

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')
#drive.mount("/content/gdrive", force_remount=True)

import sys
root_path = '/content/gdrive/My Drive/Sapienza/NLP/HM1/Arci/'  #change dir to your project folder
sys.path.insert(0, root_path)

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
import numpy as np
import tensorflow as tf
import tensorflow.keras as K
from tensorflow.keras.layers import Input, Embedding, Bidirectional, LSTM, Dense, TimeDistributed, concatenate, Activation, Masking, Dropout
from tensorflow.keras.models import load_model
import pickle
import os, datetime, time

import ChinesePreprocess as CP
import ModelConfiguration as ModelConfig
import TrainingUtils
from code_provided.score import score
import itertools

# Preprocess Initialization
For all the general cases here, the reordered datasets will be used, and the vocabulary according to each dataset. 

In [0]:
# Model to TPU (Only run when TPU is activated)
TPU_WORKER = 'grpc://' + os.environ['COLAB_TPU_ADDR']
tf.logging.set_verbosity(tf.logging.INFO)

def toTPU(mdl):
  tpu_model = tf.contrib.tpu.keras_to_tpu_model(mdl,strategy=tf.contrib.tpu.TPUDistributionStrategy(tf.contrib.cluster_resolver.TPUClusterResolver(TPU_WORKER)))
  return tpu_model

The default dataset will be MSR, unless especified different.

In [0]:
# Initialization
models = {}
modelstpu = {}
hists = {}

def load_defaults_saved_all():
  with open(root_path + "../data_load/xinput_all_100.pkl",'rb') as file:
    xinput = pickle.load(file)
  with open(root_path + "../data_load/yout_all_100.pkl",'rb') as file:
    yout = pickle.load(file)
  with open(root_path + "../vocabs/vocab_all.pkl",'rb') as file:
    vocab = pickle.load(file)
  with open(root_path + "../data_load/xtest_all_100.pkl",'rb') as voc:
    xtest = pickle.load(voc)
  with open(root_path + "../data_load/ytest_all_100.pkl",'rb') as file:
    ytest = pickle.load(file)
    #print("Vocabulary loaded.")
  
  return xinput, yout, xtest, ytest, vocab

#-------------------------------------------
########### Callbacks when fitting #########
#-------------------------------------------

def get_callbacks(model_name):
  tf_callback = K.callbacks.TensorBoard(log_dir=root_path+'logs/'+model_name)
  time_callback = TrainingUtils.TimeHistory()
  csv_logger = K.callbacks.CSVLogger(root_path+'../models/'+model_name+'.log')
  checkpoint = K.callbacks.ModelCheckpoint(root_path+'../models/'+model_name+"_weights-chkp-{epoch:02d}-{val_acc:.2f}.hdf5", monitor='val_acc', verbose=1, save_best_only=True, mode='max')

  #earlyStop_callback= K.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=0, verbose=0, mode='auto', baseline=None, restore_best_weights=False)

  return tf_callback, time_callback, csv_logger, checkpoint#, earlyStop_callback

class train_config():
  def __init__(self,tag,dataset,lenvocab,opt='adam', lr=0.001, drop_lstm=0.0, drop_rec=0.0, emb_size1=64, emb_size2=64, batch_size=256, lstm_units=256,maxlength=50):
    self.tag=tag
    self.dataset=dataset
    self.lenvocab=lenvocab
    self.opt=opt
    self.lr=lr
    self.drop_lstm=drop_lstm
    self.drop_rec=drop_rec
    self.emb_size1=emb_size1
    self.emb_size2=emb_size2
    self.batch_size=batch_size
    self.lstm_units=lstm_units
    self.maxlength=maxlength

  
def get_model_custom(TC): ## TC = Training configuration
    
    input1 =  Input(shape=(TC.maxlength,))
    uni_layer = Embedding(TC.lenvocab,TC.emb_size1, input_length=TC.maxlength, mask_zero=True)(input1)
    input2   = Input(shape=(TC.maxlength,))
    bi_layer = Embedding(TC.lenvocab,TC.emb_size2, input_length=TC.maxlength, mask_zero=True)(input2)
    ngram_layer = concatenate([uni_layer, bi_layer])
    lstm_layer = Bidirectional(LSTM(TC.lstm_units, return_sequences=True, dropout=TC.drop_lstm, recurrent_dropout=TC.drop_rec))(ngram_layer)
    time_dist_layer = TimeDistributed(Dense(4, activation='softmax'))(lstm_layer)
    model = K.models.Model([input1,input2],time_dist_layer)
    
    if tconfig.opt=='adam':
      optim = K.optimizers.Adam(lr=tconfig.lr)
    else:
      optim = K.optimizers.Adam(lr=tconfig.lr)
    
    model.compile(optimizer=optim, loss='categorical_crossentropy', metrics = ['acc', tf.keras.metrics.Recall(), tf.keras.metrics.Precision()])
    return model
  
def get_model_name(tc):
  model_name = datetime.datetime.fromtimestamp(time.time()).strftime('%Y%m%d_')+"%s_%s_%s_lr%s_drLS%s_drRec%s_emb1%s_emb2%s_batch%s_units%s_maxl%s" % (str(tc.tag),str(tc.dataset), tc.opt, str(tc.lr), str(tc.drop_lstm), str(tc.drop_rec), str(tc.emb_size1), str(tc.emb_size2), str(tc.batch_size), str(tc.lstm_units), str(tc.maxlength) )
  return model_name

## Preprocessing all. Save useful variables

In [0]:
def load_defaults_all(trainds="ALL_training_simp_reordered_shuf", testds="ALL_test_gold_simp_reordered_shuf", maxlength = 50, vocab_path = "vocab_all", samples_porcent =  1, verbose=False):
  print("Loading configurations.")
  print("Training dataset: ", trainds)
  print("Testing dataset: ", testds)
  print("Vocabulary: ", vocab_path)
  print("maxlength: ", maxlength)
  
  print("\n******************** Loading vocabulary **********************\n")
  with open(root_path + "../vocabs/%s.pkl" % vocab_path,'rb') as voc:
    vocab = pickle.load(voc)
    print("Vocabulary loaded.")
  
  print("\n******************** Extract train data (ALL) **********************\n")
  ds_train = CP.ChinesePreprocess(root_path + "../dataset/icwb2-data/training/%s.utf8" % trainds, 
                                  num_samples=0, # Should be zero mostly. Becasue it's convenient to read everything most of the times. If not zero, some words to build the vocabulary might get lost in the trimming
                                  vocabulary = vocab, # Static preset vocabulary. Same in training
                                  verbose=False)

  print("\n\n******************** Extract test data (ALL) *******************\n")
  ds_test = CP.ChinesePreprocess(root_path + "../dataset/icwb2-data/gold/%s.utf8" % testds, 
                                 num_samples=0, 
                                 vocabulary = vocab,# Same vocabulary of training
                                 verbose=False)

    # Maximum length for padding
  train = CP.ChinesePreprocess.apply_padding_data_and_labels(ds_train, maxlength, False)
  test = CP.ChinesePreprocess.apply_padding_data_and_labels(ds_test, maxlength, False)
  
   # 100% of the samples
  print("Percentage taken: ", samples_porcent*100,'%')
  
  # This is for trainings only. Not for testing 
  k = round(len(train.unigrams_pad)*samples_porcent)
  print("Number of samples: ", k)
  xinput = [train.unigrams_pad[:k], train.bigrams_pad[:k]]
  yout = train.labels[:k]
  print("Done.")
  return train, test, xinput, yout, vocab


In [0]:
#train, test, xinput, yout, vocab = load_defaults_all(maxlength=50,samples_porcent=1)
#with open(root_path+"../data_load/xinput_all_100.pkl", 'wb') as file:
#  pickle.dump(xinput, file)
#with open(root_path+"../data_load/yout_all_100.pkl", 'wb') as file:
#  pickle.dump(yout, file)
#with open(root_path+"../data_load/xtest_all_100.pkl", 'wb') as file:
#  pickle.dump([test.unigrams_pad, test.bigrams_pad], file)
#with open(root_path+"../data_load/ytest_all_100.pkl", 'wb') as file:
#  pickle.dump(test.labels, file)
#with open(root_path+"../data_load/vocab_all.pkl", 'wb') as file:
# pickle.dump(vocab, file)

Loading configurations.
Training dataset:  ALL_training_simp_reordered_shuf
Testing dataset:  ALL_test_gold_simp_reordered_shuf
Vocabulary:  vocab_all
maxlength:  50

******************** Loading vocabulary **********************

Vocabulary loaded.

******************** Extract train data (ALL) **********************

[INFO] Using preset vocabulary. No. of elements:  1043604


******************** Extract test data (ALL) *******************

[INFO] Using preset vocabulary. No. of elements:  1043604
Percentage taken:  100 %
Number of samples:  336430
Done.


In [0]:
len(train.bigrams_pad)

336430

array([907116, 124420, 333878, 955007,  79251, 879095, 843694, 907757,
       637562, 127326, 869954, 188286, 187659, 333878, 530873, 569864,
       869954, 992885, 188286, 217176, 897331, 217176, 955007, 204905,
       992885, 188286, 333878,  79251, 702761, 835892,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0], dtype=int32)

# Grid search 2 (All)
Based on the experiments with the subsets, a low learning rate, around 0.0005, and a dropout around 0.4 can show good results.
We will try that first, and then, if there's time, change the learning rate. We will go for 50 epochs.


## Training 2.1.1: LR:0.0005, Drop_LSTM=0.4, Drop_Rec=0.4
|   Static   | Variable                    |
|:----------:|-----------------------------|
|  Drop_LSTM = [0.4]   | LR=0.0005, 0.00025 |

In [0]:
xinput, yout, xtest, ytest, vocab = load_defaults_saved_all()
#x test is a list of unigram and bigrams like this [unigrams, bigrams]

In [0]:
learn_rate=0.0005
dropout_LSTM = 0.4
dropout_REC = 0.4
print("*********** Training started on: ", datetime.datetime.fromtimestamp(time.time()).strftime('%Y-%m-%d - %H:%M:%S'))

maxlength = 50
tconfig = train_config(tag="_Final2.1.1(GPU)", dataset='all100', lenvocab=len(vocab), opt='adam', lr=learn_rate, drop_lstm=dropout_LSTM, drop_rec=dropout_REC, emb_size1=64, emb_size2=64, batch_size=512, lstm_units=256,maxlength=50)
print("Configuration: ")
print(tconfig.__dict__)  
model = get_model_custom(tconfig)
#model = toTPU(model)
model_name = get_model_name(tconfig)
print("Model name: " + model_name)
#models[id].summary()
tf_callback, time_callback, csv_logger, checkpoint = get_callbacks(model_name)
start = time.time()
hist = model.fit(xinput, yout, 
                         validation_data=(xtest,ytest),
                         epochs=50,
                         batch_size=tconfig.batch_size,
                         callbacks=[tf_callback, time_callback, csv_logger, checkpoint],
                        )

end = time.time()-start
print("*********** Training finished on: ", datetime.datetime.fromtimestamp(time.time()).strftime('%Y-%m-%d - %H:%M:%S'))
print("Elapsed time: ", str(end), " seconds (", str(end/60), " minutes).")
H = hist
TrainingUtils.save_model_results(model,hist,root_path,model_name)
print("Done.\n\n")

*********** Training started on:  2019-04-24 - 17:43:18
Configuration: 
{'tag': '_Final2.1.1(GPU)', 'dataset': 'all100', 'lenvocab': 1043604, 'opt': 'adam', 'lr': 0.0005, 'drop_lstm': 0.4, 'drop_rec': 0.4, 'emb_size1': 64, 'emb_size2': 64, 'batch_size': 512, 'lstm_units': 256, 'maxlength': 50}
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Model name: 20190424__Final2.1.1(GPU)_all100_adam_lr0.0005_drLS0.4_drRec0.4_emb164_emb264_batch512_units256_maxl50
Train on 336430 samples, validate on 12590 samples
Instructions for updating:
Use tf.cast instead.
Epoch 1/50
336384/336430 [============================>.] - ETA: 0s - loss: 0.3517 - acc: 0.8605 - recall: 0.7935 - precision: 0.9294
Epoch 00001: val_acc improved from -inf to 0.92360, saving model to /content/gdrive/My Drive/Sapienza/NLP/HM1/Arci/../models/20190424__Final2.1.1(GPU)_all100_adam_lr0.000

KeyboardInterrupt: ignored